In [1]:
import numpy as np
from scipy import special as sp
import matplotlib.pyplot as plt

In [42]:
import sys
if "../src/" not in sys.path:
    sys.path.insert(0,"../src/")
from pbcpy.grid import DirectGrid, ReciprocalGrid
from pbcpy.field import DirectField, ReciprocalField
from pbcpy.functionals import *
from pbcpy.local_functionals_utils import HartreePotentialReciprocalSpace



%matplotlib inline
%load_ext autoreload
%autoreload 2
#
from pbcpy.formats.qepp import PP
from pbcpy.formats.xsf import XSF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
mol = PP(filepp='Al_fde_rho.pp').read()
rho_of_r = mol.field
grid=mol.cell

PP_file not set in input. Can do so manually invoking Atom.local_PP
PP_file not set in input. Can do so manually invoking Atom.local_PP


In [4]:
precision = 1.0e-8

# Third term

$$
\sum_{ix,iy,iz=-N_x,-N_y,-N_z}^{N_x,N_y,N_z} \sum_{i,j}
$$

In [5]:
L=np.sqrt(np.einsum('ij->j',grid.lattice**2))
L

array([  5.40641961,   5.40641961,  10.81283921])

In [6]:
gmax=0.0
gmax_x = np.amax(grid.get_reciprocal().gg[:,0,0])
gmax_y = np.amax(grid.get_reciprocal().gg[0,:,0])
gmax_z = np.amax(grid.get_reciprocal().gg[0,0,:])
gmax = np.amin([gmax_x,gmax_y,gmax_z])/20.0
print([gmax_x,gmax_y,gmax_z])
print(gmax)

[656.41199309834917, 656.41199309834917, 656.41199309834917]
32.8205996549


In [7]:
# This was good, Pablo!

charge = 0.0
for i in np.arange(mol.natoms):
    charge+=mol.ions[i].Zval
print('Charge = ',charge)
NotGoodEta = True
eta = 1.2
while NotGoodEta:
    upbound = 2.0 * charge**2 *np.sqrt(eta/np.pi)*sp.erfc(np.sqrt((gmax)/4.0/eta))
    print(upbound)
    if upbound<precision:
        NotGoodEta = False
    else:
        eta = eta - 0.1
        
print(eta)

Charge =  6.0
0.00966956808083
0.00478274043146
0.00207200114244
0.000752992683233
0.000215206452265
4.37239740604e-05
5.34032505313e-06
2.90306517085e-07
3.85979067337e-09
0.4


In [8]:
L=np.sqrt(np.einsum('ij->j',grid.lattice**2))
rmax = np.array([4.0 / np.sqrt(eta),4.0 / np.sqrt(eta),4.0 / np.sqrt(eta)])
print(4.0 / np.sqrt(eta))
N=np.rint(rmax/L)
print(N)

6.32455532034
[ 1.  1.  1.]


In [9]:
sum = np.float(0.0)
for ix in np.arange(-N[0]+1,N[0]):
    for iy in np.arange(-N[1]+1,N[1]):
        for iz in np.arange(-N[2]+1,N[2]):
            R=np.einsum('j,ij->i',np.array([ix,iy,iz],dtype=np.float),grid.lattice.transpose())
            #print('R = ',R)
            for i in np.arange(mol.natoms):
                for j in np.arange(i+1):
                    if j == i:
                        if np.dot(R,R.transpose()) < precision:
                            break
                    Zi=mol.ions[i].Zval
                    Zj=mol.ions[j].Zval
                    rij = mol.ions[i].pos.d_mic(mol.ions[j].pos)-R
                    dij=rij.length()
                    sum += Zi*Zj*sp.erfc(np.sqrt(eta)*dij)/dij
                    #print(dij,sum)
third=sum
print(third)

2.20924229208e-06


# Second term

In [10]:
reciprocal_grid = grid.get_reciprocal()
gg=reciprocal_grid.gg
strf = mol.ions[0].strf(reciprocal_grid) * mol.ions[0].Zval
for i in np.arange(1,mol.natoms):
    strf += mol.ions[i].strf(reciprocal_grid) * mol.ions[i].Zval
strf_sq =np.conjugate(strf)*strf
gg[0,0,0,0]=1
invgg=1.0/gg
invgg[0,0,0,0]=0.0
second=np.real(4.0*np.pi*np.sum(strf_sq*np.exp(-gg/(4.0*eta))*invgg) / grid.volume)
print(second)

3.31701671831


# Fourth term

In [11]:
const=-np.sqrt(8.0/np.pi*eta)
sum = np.float(0.0)
for i in np.arange(mol.natoms):
    sum+=mol.ions[i].Zval**2
fourth=const*sum 
print(fourth)

-18.1665541585


# Fifth term

In [12]:
const=-4.0*np.pi*(1.0/(4.0*eta*grid.volume))
sum = np.float(0.0)
for i in np.arange(mol.natoms):
    sum+=mol.ions[i].Zval
fifth=const*sum**2
print(fifth)

-1.26516781812


# First term

In [13]:
from pbcpy.hartree import HartreeFunctional
Hartree = HartreeFunctional(rho_of_r)
first=-Hartree.energydensity.integral()
print(first)

-0.00839883264471


In [14]:
second+third+fourth+fifth

-16.114703049111057

In [15]:
from pbcpy.local_pseudopotential import NuclearElectron

In [16]:
Nucel = NuclearElectron(mol.ions,rho_of_r,['./Al_lda.oe01.recpot','./Al_lda.oe01.recpot'])

Recpot pseudopotential ./Al_lda.oe01.recpot loaded
Recpot pseudopotential ./Al_lda.oe01.recpot loaded


# Alpha Z term

In [17]:
alpha = 0.0
for i in range(len(mol.ions)):
    alpha +=mol.ions[i].alpha_mu

In [18]:
Z = 0.0
for i in range(len(mol.ions)):
    Z +=mol.ions[i].Z

In [19]:
alpha_z = alpha * Z / mol.cell.volume
print(alpha_z)

6.06843916205


In [21]:
second+third+fourth+fifth+alpha_z

-10.046263887060082

In [45]:
v_h = HartreePotentialReciprocalSpace(density=rho_of_r)

In [46]:
vloc = np.zeros_like(rho_of_r.fft())
for i in range(len(mol.ions)):
    vloc+=mol.ions[i].v

In [47]:
vloc[0,0,0,0] = 0.0

In [60]:
E_loc =  np.real(np.sum((vloc + v_h)* np.conj(rho_of_r.fft()) ))* mol.cell.volume
print(E_loc)

-4826.804195782919


In [49]:
vloc[0,0,0,0]

0j

In [50]:
v_h[0,0,0,0]

0j